In [ ]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py39_4.9.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py39" --user

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
!unzip -q "/content/gdrive/MyDrive/Inputs.zip"

Mounted at /content/gdrive


In [ ]:
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
# %pip install torchsampler
import matplotlib.pyplot as plt
from pandas.core.common import flatten  
import numpy as np
import random
import torch
import os
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, Subset
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2


In [ ]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")



In [ ]:
p = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0 ,0 , 1, 1, 1, 1, 1, 1, 0, 0, 0])
p = torch.from_numpy(p)
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir,  transform=None):
        self.img_labels = annotations_file
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image_matrix = np.load(os.path.join('Inputs/Pot_{}_Input_Seq.npy'.format(idx+1 if idx>=9 else '0'+str(idx+1))), allow_pickle=True)
        # image_matrix=np.vstack(image_matrix).astype(np.float64)
        image_matrix=np.array(image_matrix, dtype=[('O', np.float64)]).astype(np.float64)
        label = self.img_labels[idx]

        cropped_img_matrix = []
        if self.transform is not None:
          for image in image_matrix:
            # image = self.transform(image=image)["image"]
            # print(self.transform(image=image)["image"].shape)
            cropped_img_matrix.append(self.transform(image=image)["image"])
        my_tensor = torch.stack(cropped_img_matrix)
        # image_matrix = torch.from_numpy(image_matrix)

        
        return my_tensor, label

val_transform = A.Compose(
    [
        A.CenterCrop(height=224, width=224),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.9),
        # A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)



transformed_dataset = CustomImageDataset(annotations_file=p, img_dir='Inputs', transform=val_transform)

def weight_rand(images):
  n_images = len(images)
  count_per_class = [0] * 2
  for _,image_class in images:
      count_per_class[image_class] += 1
  weight_per_class = [0.] * 2
  for i in range(2):
      weight_per_class[i] = float(n_images) / float(count_per_class[i])
  weights = [0] * n_images
  for idx, (imgggg, image_class) in enumerate(images):
      weights[idx] = weight_per_class[image_class]
  # print(weight_per_class, count_per_class)

  return weights, weight_per_class, count_per_class




In [ ]:
train_dataset = transformed_dataset

In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(25, 5))


# for count, (imgs, labels) in enumerate(train_dataset):
#   print(imgs.shape)
#   for i, img in enumerate(imgs):
#         if i == 15:
#           plt.imshow(img[0])
#           break
#         # print(img.shape)

        
#   if count == 1:
#     break

In [ ]:

# train_size = int(0.8 * len(transformed_dataset))
# test_size = len(transformed_dataset) - train_size
# train_dataset, test_dataset = torch.utils.data.random_split(transformed_dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))

In [ ]:
# # num_samples = 16
# # fig, ax = plt.subplots(50, num_samples, figsize=(25, 5))

# # for imgs, labels in train_dataset:
# #     # ax[i].imshow(transforms.ToPILImage()(train_dataset[0][0]))
# #   for img in imgs:
# #     ax[i].imshow(transforms.ToPILImage()(img))
# #     ax[i].axis('off')
# #   print(lables)
# fig, ax = plt.subplots(2, 16, figsize=(25, 5))


# for count, (imgs, labels) in enumerate(train_dataset):
#   print(imgs.shape)
#   for i, img in enumerate(imgs):
#         ax[count, i].imshow(img[0])
#         # print(img.shape)

#   if count == 1:
#     break

In [ ]:
# p, q, r = weight_rand(train_dataset)
# print(p)
# print(q)
# print(r)

In [ ]:
# p = [3.4285714285714284, 1.411764705882353, 1.411764705882353, 1.411764705882353, 1.411764705882353, 1.411764705882353, 1.411764705882353, 3.4285714285714284, 1.411764705882353, 1.411764705882353, 3.4285714285714284, 1.411764705882353, 3.4285714285714284, 1.411764705882353, 1.411764705882353, 1.411764705882353, 3.4285714285714284, 3.4285714285714284, 3.4285714285714284, 1.411764705882353, 1.411764705882353, 1.411764705882353, 1.411764705882353, 1.411764705882353]

In [ ]:
# len(p)

In [ ]:
# train_dataset = DataLoader(train_dataset, sampler=WeightedRandomSampler(weights = p,
                                  # num_samples = 50, replacement=True))
# test_dataset = DataLoader(test_dataset, sampler=WeightedRandomSampler(weights=weight_rand(test_dataset),
#                                   num_samples = len(test_dataset), replacement=True))

In [ ]:
len(train_dataset)

30

In [ ]:
# fig, ax = plt.subplots(2, 16, figsize=(25, 5))


# for count, (imgs, labels) in enumerate(train_dataset):
#   for i, img in enumerate(imgs):
#         ax[count, i].imshow(img[0].T)
#         print(img.shape)

#   if count == 1:
#     break

In [ ]:
# # plt.subplot(2, 3, 1)
# # for count, (imgs, labels) in enumerate(train_dataset):
# #       # num_samples = 16
# #       # fig, ax = plt.subplots(1, num_samples, figsize=(25, 5))
# #       for i, img in enumerate(imgs):
# #         # ax[i].imshow(transforms.ToPILImage()(img[0]))
# #         plt.subplot(2, 16, i+1)

# #         # ax[i].axis('off')
# #         plt.imshow(img[0].permute(1, 2, 0))


# #       if count == 1:
# #         break


# for count, (imgs, labels) in enumerate(train_dataset):
#   for img in imgs:
#         plt.imshow(img[0].permute(1, 2, 0))

#   if count == 0:
#     break



In [ ]:
# num_samples = 16
# fig, ax = plt.subplots(50, num_samples, figsize=(25, 5))

# for imgs, labels in train_dataset:
#     # ax[i].imshow(transforms.ToPILImage()(train_dataset[0][0]))
#   for img in imgs:
#     ax[i].imshow(transforms.ToPILImage()(img))
#     ax[i].axis('off')
#   print(lables)


In [ ]:
# num_samples = 16
# fig, ax = plt.subplots(1, num_samples, figsize=(25, 5))
# for i in range(num_samples):
#   # ax[i].imshow(transforms.ToPILImage()(train_dataset[0][0]))
#   for img in train_dataset[0][0]:
#     ax[i].imshow(transforms.ToPILImage()(img))
#     ax[i].axis('off')

In [ ]:
# A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.9)

In [ ]:
len(train_dataset)

30

In [ ]:
class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias, device):
        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias
        self.device = device
        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state
        h_cur, c_cur = h_cur.to(self.device), c_cur.to(self.device)
        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width).to(self.device),
                torch.zeros(batch_size, self.hidden_dim, height, width).to(self.device))
        
class ConvLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size,device,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []

        cur_input_dim = self.input_dim 
        self.sigmoid = nn.Sigmoid()
        self.device = device


        self.conv_lstm = ConvLSTMCell(input_dim=cur_input_dim,
                                      hidden_dim=self.hidden_dim,
                                      kernel_size=self.kernel_size,
                                          bias=self.bias, device = self.device)
        
        self.conv1 = nn.Conv2d(self.hidden_dim, 20, 3)
        self.pool1 = nn.MaxPool2d(2, 2)             
        self.conv2 = nn.Conv2d(20, 40, 3)       
        self.conv3 = nn.Conv2d(40, 60, 3)       
        self.conv4 = nn.Conv2d(60, 80, 3)       
        self.fc1 = nn.Linear(80*12*12, 3000)
        self.fc2 = nn.Linear(3000, 700)
        self.fc3 = nn.Linear(700, 150)
        self.fc4 = nn.Linear(150, 50)
        self.fc5 = nn.Linear(50, 1)
    def forward(self, input_tensor):
        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        h, c = self._init_hidden(input_tensor.size(0), (input_tensor.size(3), input_tensor.size(4)))
        h,c = h.to(self.device), c.to(self.device)
        for t in range(seq_len):
            h, c = self.conv_lstm(input_tensor=cur_layer_input[:, t, :, :, :],
                                              cur_state=[h, c])
            
        y = h
        y = self.pool1(F.relu(self.conv1(y))) 
        y = self.pool1(F.relu(self.conv2(y))) 
        y = self.pool1(F.relu(self.conv3(y)))
        y = self.pool1(F.relu(self.conv4(y)))
        # print(y.shape, 80*12*12)
        out = y.view(-1, 80*12*12)
        # print(out.shape)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = F.relu(self.fc3(out))
        out = F.relu(self.fc4(out))
        out = self.sigmoid(self.fc5(out))
        return out[0]
        out = out.view(1, -1)
        out = out[:,-1]
        # print(out[0][0][0].shape)
        return out[0][0][0]       # out, hidden = self.lstm1(y.view(-1, 10920), hidden)                     
        # out = F.relu(self.fc1(out))

        # out = self.sigmoid(self.fc6(out))
        # out = out.view(1, -1)
        # out = out[:,-1]
        # return out
            
        # return h

    def _init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width).to(self.device),
                torch.zeros(batch_size, self.hidden_dim, height, width).to(self.device))

In [ ]:
input_dim, hidden_dim, kernel_size = 3, 10, (3,3)
model_conv = ConvLSTM(input_dim, hidden_dim, kernel_size, device)
model_conv = model_conv.to(device)

In [ ]:
lr=0.0001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_conv.parameters(), lr=lr)

In [ ]:
# fig, ax = plt.subplots(12, 16, figsize=(25, 5))

my_subset = Subset(train_dataset, [0, 1, 2, 3, 4, 5, 12, 13 ,14, 18, 19, 20] )
# for count, (imgs, labels) in enumerate(my_subset):
#   # print(imgs.shape)
#   for i, img in enumerate(imgs):
#         ax[count, i].imshow(img[0])
#         # print(img.shape)

#   if count == 12:
#     break

In [ ]:
no_of_epochs = 10
counter = 0
loss_list = []

# model.train()
for epoch in range(no_of_epochs):
    # running_loss = 0.0
    print('-------------------Epoch_{}----------------------------------'.format(epoch))

    # for inputs, labels in train_dataset:
    for index, (inputs, labels) in enumerate(my_subset):
          print('-------------------------------------------------------------')
          inputs, labels = inputs.to(device), labels.to(device)
          counter += 1
          model_conv.zero_grad()
          output = model_conv(inputs.unsqueeze(0))
          print('Pot',index)
          print('Input Shape: ', inputs.unsqueeze(0).shape)
          print('Pot: ', output)

          print('Predicted Sigmoid',output.squeeze().item(),'Truth Label', labels.item())
          print(labels.float().shape)
          loss = criterion(output.squeeze(), labels.float())
          loss.backward()
          optimizer.step()
          print('Loss', loss.item())
          loss_list.append(loss.item())
          # if epoch == 100 or epoch == 200 or epoch == 300 or epoch == 400 or epoch == 500 or epoch == 50 :
          #   torch.save(model_conv.state_dict(), './state_dict_{}.pt'.format(epoch))
          #   np.savez("./losslist_{}".format(epoch), np.array(loss_list))
          print('------------------------------------------------------')
          

print('done')

-------------------Epoch_0----------------------------------
-------------------------------------------------------------
Pot 0
Input Shape:  torch.Size([1, 16, 3, 224, 224])
Pot:  tensor([0.4783], device='cuda:0', grad_fn=<SelectBackward0>)
Predicted Sigmoid 0.4783414900302887 Truth Label 1
torch.Size([])
Loss 0.7374303936958313
------------------------------------------------------
-------------------------------------------------------------
Pot 1
Input Shape:  torch.Size([1, 16, 3, 224, 224])
Pot:  tensor([0.4810], device='cuda:0', grad_fn=<SelectBackward0>)
Predicted Sigmoid 0.48099541664123535 Truth Label 1
torch.Size([])
Loss 0.7318975329399109
------------------------------------------------------
-------------------------------------------------------------
Pot 2
Input Shape:  torch.Size([1, 16, 3, 224, 224])
Pot:  tensor([0.4828], device='cuda:0', grad_fn=<SelectBackward0>)
Predicted Sigmoid 0.48280709981918335 Truth Label 1
torch.Size([])
Loss 0.7281380891799927
-----------

In [ ]:
# # Loading the best model
# model.load_state_dict(torch.load('./state_dict.pt'))
num_correct = 0
# model.eval()
test_dataset = Subset(train_dataset,[6, 7, 8, 9, 10, 11, 15, 16, 17, 21, 22, 23, 24, 25, 26, 27, 28, 29]) 
with torch.no_grad():
  for inputs, labels in test_dataset:
      print('-----------------------------------------------------')
      inputs, labels = inputs.to(device), labels.to(device)
      output = model_conv(inputs.unsqueeze(0))
      # test_loss = criterion(output.squeeze(), labels.float())
      # print('Test Loss : ', test_loss)
      print('Predicted Sigmoid: ', output.squeeze().item())

      pred = torch.round(output.squeeze())  # Rounds the output to 0/1
      correct_tensor = pred.eq(labels.float().view_as(pred))
      print('Predicted Label: ', pred)
      print('Truth: ', labels)
      # model.zero_grad()
      if int(pred.item()) == labels:
        num_correct += 1 

print("accuracy: ", num_correct/18)

-----------------------------------------------------
Predicted Sigmoid:  0.4869213402271271
Predicted Label:  tensor(0., device='cuda:0')
Truth:  tensor(1, device='cuda:0')
-----------------------------------------------------
Predicted Sigmoid:  0.4869213402271271
Predicted Label:  tensor(0., device='cuda:0')
Truth:  tensor(1, device='cuda:0')
-----------------------------------------------------
Predicted Sigmoid:  0.4869212806224823
Predicted Label:  tensor(0., device='cuda:0')
Truth:  tensor(1, device='cuda:0')
-----------------------------------------------------
Predicted Sigmoid:  0.4869213402271271
Predicted Label:  tensor(0., device='cuda:0')
Truth:  tensor(1, device='cuda:0')
-----------------------------------------------------
Predicted Sigmoid:  0.4869212806224823
Predicted Label:  tensor(0., device='cuda:0')
Truth:  tensor(1, device='cuda:0')
-----------------------------------------------------
Predicted Sigmoid:  0.4869212806224823
Predicted Label:  tensor(0., device='